In [1]:
import os

In [2]:
%pwd

'd:\\End-to-End-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\End-to-End-ML-Project'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/MaheshkumarGovind/End-to-End-ML-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "MaheshkumarGovind"
os.environ["MLFLOW_TRACKING_PASSWORD"]="fa77d4d80e2686e48fdc5485a5fbd0b921f10362"


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/MaheshkumarGovind/End-to-End-ML-Project.mlflow",
           
        )

        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    

In [ ]:
import requests
import os

# Replace with your repository URL and credentials from environment variables
repo_url = os.getenv('MLFLOW_TRACKING_URI')
username = os.getenv('MLFLOW_TRACKING_USERNAME')
password = os.getenv('MLFLOW_TRACKING_PASSWORD')

response = requests.get(repo_url, auth=(username, password))

if response.status_code == 200:
    print("Authentication successful!")
else:
    print("Authentication failed. Status Code:", response.status_code)
    print("Response:", response.text)


In [ ]:
import os
import mlflow
import requests

# Replace with your repository URL and credentials from environment variables
repo_url = os.getenv('MLFLOW_TRACKING_URI')
api_token = os.getenv('MLFLOW_TRACKING_PASSWORD')

headers = {
    'Authorization': f'Token {api_token}'
}

# Verify authentication with DagsHub using the API token
response = requests.get(repo_url, headers=headers)

if response.status_code == 200:
    print("Authentication successful!")
else:
    print("Authentication failed. Status Code:", response.status_code)
    print("Response:", response.text)

# Set MLflow tracking URI
mlflow.set_tracking_uri(repo_url)
mlflow.set_registry_uri(repo_url)

# Start MLflow run
try:
    with mlflow.start_run():
        print("MLflow run started successfully")
        # Your code for model evaluation and logging goes here
        mlflow.log_param("param1", 5)
        mlflow.log_metric("metric1", 0.85)
except Exception as e:
    print("Error starting MLflow run:", e)


In [ ]:
import os
import mlflow

# Set MLflow tracking URI directly with token in URI
mlflow.set_tracking_uri(f"{os.getenv('MLFLOW_TRACKING_URI')}?token={os.getenv('MLFLOW_TRACKING_PASSWORD')}")

# Start MLflow run
try:
    with mlflow.start_run():
        print("MLflow run started successfully")
        # Your code for model evaluation and logging goes here
        mlflow.log_param("param1", 5)
        mlflow.log_metric("metric1", 0.85)
except Exception as e:
    print("Error starting MLflow run:", e)


In [15]:
import os
import mlflow

# Set MLflow tracking URI directly with token in URI
mlflow.set_tracking_uri(f"{os.getenv('MLFLOW_TRACKING_URI')}?token={os.getenv('MLFLOW_TRACKING_PASSWORD')}")

try:
    with mlflow.start_run():
        print("MLflow run started successfully")
        # Your code for model evaluation and logging goes here
        mlflow.log_param("param1", 5)
        mlflow.log_metric("metric1", 0.85)
except Exception as e:
    print("Error starting MLflow run:", e)


Error starting MLflow run: [WinError 3] The system cannot find the path specified: 'S:\\'


In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-07-02 11:34:42,434: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-02 11:34:42,447: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-02 11:34:42,472: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-02 11:34:42,481: INFO: common: created directory at: artifacts]
[2024-07-02 11:34:42,489: INFO: common: created directory at: artifacts/model_evaluation]


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'S:\\'